In [ ]:
#
# Stub Bundling and Confluent Spirals for Geographic Networks
# - Arlind Nocaj and Ulrik Brandes
# - Department of Computer & Information Science, University of Konstanz
# S. Wismath and A. Wolff (Eds.): GD 2013, LNCS 8242, pp. 388–399, 2013.
#
from math import pi, tan, exp, cos, sin, acos
import rtsvg
rt = rtsvg.RACETrack()
x0, y0, x1, y1 = -20.0, -20.0, 20.0, 20.0
ws, hs         = 512, 512
pw    = (10.0,  -8.0)
pv    = ( 6.0,   5.0)
svg = [f'<svg x="0" y="0" width="{ws}" height="{hs}" viewBox="{x0} {y0} {x1-x0} {y1-y0}" xmlns="http://www.w3.org/2000/svg">']
svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="#ffffff" />')
x = int(x0)
while x < x1:
    svg.append(f'<line x1="{x}" y1="{y0}" x2="{x}" y2="{y1}" stroke="#a0a0a0" stroke-width="0.05"/>')
    x += 1
y = int(y0)
while y < y1:
    svg.append(f'<line x1="{x0}" y1="{y}" x2="{x1}" y2="{y}" stroke="#a0a0a0" stroke-width="0.05"/>')
    y += 1
svg.append(f'<line x1="{0.0}" y1="{y0}"  x2="{0.0}" y2="{y1}"  stroke="#a0a0a0" stroke-width="0.1"/>')
svg.append(f'<line x1="{x0}"  y1="{0.0}" x2="{x1}"  y2="{0.0}" stroke="#a0a0a0" stroke-width="0.1"/>')
svg.append(rt.svgText('pw', pw[0], pw[1]-0.5, txt_h=1.5, color='#a0a0a0', anchor='middle'))
svg.append(f'<circle cx="{pw[0]}" cy="{pw[1]}" r="0.2" fill="#ff0000"/>')
svg.append(rt.svgText('pv', pv[0], pv[1]+2.0, txt_h=1.5, color='#a0a0a0', anchor='middle'))
svg.append(f'<circle cx="{pv[0]}" cy="{pv[1]}" r="0.2" fill="#ff0000"/>')
svg_base = ''.join(svg)

In [ ]:
#
# Figure 4(b) from the paper / not correct still
#
from math import pi, tan, exp, cos, sin, acos
cot   = lambda x: 1.0 / tan(x)
theta = pi / 3.0
b     = cot(theta)
pw    = (10.0,  -8.0)
pv    = ( 6.0,   5.0)
def S(t):
    _in_exp_     = -abs(b) * t
    _uv_         = pv[0] - pw[0], pv[1] - pw[1]   # should be a non-unit vector
    _phi0_       = _uv_[0] * 1.0 + _uv_[1] * 0.0  # does this change for x & y?
    _in_         = (b / abs(b)) * (_phi0_ + t)
    x = pw[0] + abs(pv[0] - pw[0]) * exp(_in_exp_) * cos(_in_) # should the cos be within the exp?
    y = pw[1] + abs(pv[1] - pw[1]) * exp(_in_exp_) * sin(_in_) # should the sin be within the exp?
    return (x, y)
t      = 0.0
pts    = []
while t < 8.0:
    x,y = S(t)
    pts.append((x,y))
    t   += 0.1

In [ ]:
from math import pi, tan, exp, cos, sin, sqrt, atan2 # Added sqrt and atan2

# Define cotangent function
cot = lambda x: 1.0 / tan(x)

# Parameters from the user's code
theta = pi / 6.0      # Angle theta for calculating b
b     = cot(theta)    # b = cot(theta). Since theta is pi/3, b is positive (approx 0.577)
pw    = (10.0,  -8.0) # Point w (the pole of the spiral)
pv    = ( 6.0,   5.0) # Point v (the spiral starts here at t=0 and spirals towards w)

# --- Corrections Start Here ---

# Calculate the components of the vector u_wv = v - w
# This vector points from the pole w to the starting point v
u_wv_x = pv[0] - pw[0]   # x-component of (v - w)
u_wv_y = pv[1] - pw[1]   # y-component of (v - w)

# Calculate L0, the initial distance from w to v (magnitude of u_wv)
# L0 = ||v - w||
L0 = sqrt(u_wv_x**2 + u_wv_y**2)

# Calculate phi0, the initial angle of the vector u_wv
# This is the angle between the positive x-axis and the vector from w to v.
# math.atan2(y, x) is used to get the correct angle in all quadrants.
phi0 = atan2(u_wv_y, u_wv_x)

# --- Corrections End Here ---

# Define the spiral function S(t)
# This function calculates the (x,y) coordinates of a point on the spiral
# for a given parameter t.
# The formula used is from the paper for b > 0:
# S(t) = w + L0 * exp(-b*t) * (cos(phi0 + t), sin(phi0 + t))
def S_corrected(t):
    # Exponential decay term: exp(-b*t)
    # Since b > 0, this term causes the spiral to shrink towards the pole w as t increases.
    decay_factor = exp(-b * t)
    
    # Current angle for the cosine and sine terms: (phi0 + t)
    # phi0 is the initial phase, and t is the angle "swept" by the spiral.
    current_angle = phi0 + t
    
    # Calculate the displacement (delta_x, delta_y) from the pole pw
    # This displacement is scaled by L0 and the decay_factor.
    delta_x = L0 * decay_factor * cos(current_angle)
    delta_y = L0 * decay_factor * sin(current_angle)
    
    # Calculate the absolute coordinates of the point on the spiral
    x = pw[0] + delta_x
    y = pw[1] + delta_y
    
    return (x, y)

# Generate points for the spiral
t_current = 0.0
pts       = []
svg       = [svg_base]
# The loop iterates t from 0 up to (but not including) 8.0.
# This range determines how much of the spiral is drawn.
# S(0) will be pv, and as t increases, S(t) will approach pw.
while t_current < 3.0:
    x_coord, y_coord = S_corrected(t_current) # Use the corrected function S_corrected
    svg.append(f'<circle cx="{x_coord}" cy="{y_coord}" r="0.1" fill="#000000"/>')
    pts.append((x_coord, y_coord))
    t_current += 0.1 # Increment t to get the next point
svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
#
# ConfluentSpiral class
# - Last cell but as a rolled up class
# - Kindof looks correct... but don't understand how to align the angles to match the bundle
# ... theta can to set to (0.0, pi/3]
# ... which doesn't match the paper because the paper specifies that -pi / 2 to 0.0 can also be used...
#
class ConfluentSpiral(object):
    #
    # Constructor
    #
    def __init__(self, pv, pw, theta=pi/3.0, t_inc=0.1, t_min=0.0, t_max=8.0):
        self.pv    = pv
        self.pw    = pw
        self.theta = theta
        self.t_inc = t_inc
        self.t_min = t_min
        self.t_max = t_max
        # Define cotangent function
        cot    = lambda x: 1.0 / tan(x)
        b      = cot(theta)
        u_wv_x = pv[0] - pw[0]   # x-component of (v - w)
        u_wv_y = pv[1] - pw[1]   # y-component of (v - w)
        # Calculate L0, the initial distance from w to v (magnitude of u_wv)
        # L0 = ||v - w||
        L0 = sqrt(u_wv_x**2 + u_wv_y**2)
        # Calculate phi0, the initial angle of the vector u_wv
        # This is the angle between the positive x-axis and the vector from w to v.
        # math.atan2(y, x) is used to get the correct angle in all quadrants.
        phi0     = atan2(u_wv_y, u_wv_x)
        self.pts = []
        t        = t_min
        while t <= t_max:
            # Exponential decay term: exp(-b*t)
            # Since b > 0, this term causes the spiral to shrink towards the pole w as t increases.
            decay_factor = exp(-b * t)            
            # Current angle for the cosine and sine terms: (phi0 + t)
            # phi0 is the initial phase, and t is the angle "swept" by the spiral.
            current_angle = phi0 + t
            # Calculate the displacement (delta_x, delta_y) from the pole pw
            # This displacement is scaled by L0 and the decay_factor.
            delta_x = L0 * decay_factor * cos(current_angle)
            delta_y = L0 * decay_factor * sin(current_angle)
            # Calculate the absolute coordinates of the point on the spiral
            x = pw[0] + delta_x
            y = pw[1] + delta_y
            self.pts.append((x,y))
            t += t_inc
    #
    # SVG Representation
    #
    def _repr_svg_(self):
        x0, y0, x1, y1 = self.pts[0][0], self.pts[0][1], self.pts[0][0], self.pts[0][1]
        for _pt_ in self.pts: x0, y0, x1, y1 = min(x0,_pt_[0]), min(y0,_pt_[1]), max(x1,_pt_[0]), max(y1,_pt_[1])
        x0, y0 = min(x0, min(self.pv[0], self.pw[0])), min(y0, min(self.pv[1], self.pw[1]))
        x1, y1 = max(x1, max(self.pv[0], self.pw[0])), max(y1, max(self.pv[1], self.pw[1]))
        x_perc = 0.05 * (x1 - x0)
        y_perc = 0.05 * (y1 - y0)
        x0, y0, x1, y1 = x0 - x_perc, y0 - y_perc, x1 + x_perc, y1 + y_perc
        ws, hs = 256, 256
        svg = [f'<svg x="0" y="0" width="{ws}" height="{hs}" viewBox="{x0} {y0} {x1-x0} {y1-y0}" xmlns="http://www.w3.org/2000/svg">']
        svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="#ffffff" />')
        x = int(x0)
        if int(x1) - int(x0) < 200:
            while x < x1:
                svg.append(f'<line x1="{x}" y1="{y0}" x2="{x}" y2="{y1}" stroke="#a0a0a0" stroke-width="0.05"/>')
                x += 1
        y = int(y0)
        if int(y1) - int(y0) < 200:
            while y < y1:
                svg.append(f'<line x1="{x0}" y1="{y}" x2="{x1}" y2="{y}" stroke="#a0a0a0" stroke-width="0.05"/>')
                y += 1
        svg.append(f'<line x1="{0.0}" y1="{y0}"  x2="{0.0}" y2="{y1}"  stroke="#a0a0a0" stroke-width="0.1"/>')
        svg.append(f'<line x1="{x0}"  y1="{0.0}" x2="{x1}"  y2="{0.0}" stroke="#a0a0a0" stroke-width="0.1"/>')
        svg.append(rt.svgText('pw', self.pw[0], self.pw[1]-0.5, txt_h=1.5, color='#a0a0a0', anchor='middle'))
        svg.append(f'<circle cx="{self.pw[0]}" cy="{self.pw[1]}" r="0.2" fill="#ff0000"/>')
        svg.append(rt.svgText('pv', self.pv[0], self.pv[1]+2.0, txt_h=1.5, color='#a0a0a0', anchor='middle'))
        _path_ = [f'M {self.pts[0][0]} {self.pts[0][1]}']
        for i in range(1, len(self.pts)): _path_.append(f'L {self.pts[i][0]} {self.pts[i][1]}')
        svg.append(f'<path d="{" ".join(_path_)}" fill="none" stroke="#000000" stroke-width="0.1"/>')
        svg.append(f'<circle cx="{self.pv[0]}" cy="{self.pv[1]}" r="0.2" fill="#ff0000"/>')
        for i in range(len(self.pts)):
            _pt_ = self.pts[i]
            svg.append(f'<circle cx="{_pt_[0]}" cy="{_pt_[1]}" r="0.1" fill="#000000"/>')
        svg.append('</svg>')
        return ''.join(svg)

_pv_, _pw_ = (1.0, 2.0),(-7.0, 9.0)
rt.tile([ConfluentSpiral(_pv_, _pw_, theta=pi/2.1),
         ConfluentSpiral(_pv_, _pw_, theta=pi/3),
         ConfluentSpiral(_pv_, _pw_, theta=pi/4),
         ConfluentSpiral(_pv_, _pw_, theta=pi/5),
         ConfluentSpiral(_pv_, _pw_, theta=pi/6)], spacer=10)

In [ ]:
#
# After asking Gemini 2.5 to fix the -pi / 2 issue...
#
import math
from math import pi, exp, cos, sin, sqrt, atan2, tan, copysign # Ensure all needed functions are imported

# Define cotangent function with safety checks
def safe_cot(angle_rad):
    """
    Calculates cotangent, handling angles where tan is near zero or infinite.
    Restricts input angle to (-pi/2, pi/2) implicitly via tan behavior.
    """
    # Check if angle is very close to +/- pi/2 (tan -> +/- infinity, cot -> 0)
    if abs(abs(angle_rad) - pi/2.0) < 1e-15:
        return 0.0
        
    # Check if angle is very close to 0 (tan -> 0, cot -> +/- infinity)
    # Note: The paper's formulas assume b is finite. theta=0 is a straight line case,
    # handled separately if needed, but not as a spiral with infinite b.
    # If theta is very close to 0, tan(theta) is very small.
    tan_val = tan(angle_rad)
    if abs(tan_val) < 1e-15:
        # Return a very large number with the correct sign, or handle as error
        # Returning large number might cause issues in exp(-b*t)
        # It might be better to raise an error or warn if theta is too close to 0.
        print(f"Warning: theta={angle_rad:.4f} is very close to 0, cot(theta) approaches infinity.")
        # Return NaN or raise error? Let's return NaN for now.
        # return float('nan') 
        # Or return a large value if the calling code can handle it:
        return copysign(1e15, angle_rad) # cot(theta) has same sign as theta near 0

    # Otherwise, calculate cotangent normally
    return 1.0 / tan_val

# Define the generalized spiral function S(t)
def S_generalized(t, pw, L0, phi0, b):
    """
    Calculates the (x,y) coordinates of a point on the spiral for a given parameter t,
    handling positive, negative, and zero b.
    
    Args:
        t: Spiral parameter (related to angle swept and radial change)
        pw: Pole of the spiral (w)
        L0: Initial distance from pole to start point (||v-w||)
        phi0: Initial angle of the vector from pole to start point (angle of v-w)
        b: Spiral parameter cot(theta)
        
    Returns:
        (x, y) coordinates on the spiral.
    """
    # Case 1: b is effectively 0 (theta = +/- pi/2) -> Circle
    if abs(b) < 1e-15: # Use tolerance for floating point comparison
        # S(t) = w + L0 * (cos(phi0 + t), sin(phi0 + t))
        # The direction (+t or -t) determines clockwise/counter-clockwise.
        # Using +t is consistent with the limit from b > 0.
        current_angle = phi0 + t
        delta_x = L0 * cos(current_angle)
        delta_y = L0 * sin(current_angle)
    
    # Case 2: b is non-zero -> Logarithmic Spiral
    else:
        # Common term: L0 * exp(-b*t)
        # If b > 0, decays towards pole w.
        # If b < 0, grows away from pole w (-b is positive).
        try:
            # Calculate the radial distance factor
            radial_factor = L0 * exp(-b * t)
        except OverflowError:
            # Handle cases where exp(-b*t) becomes too large (e.g., b<0 and t large)
            print(f"Warning: Numerical overflow encountered for exp(-b*t) with b={b:.4f}, t={t:.4f}. Point set to NaN.")
            return (float('nan'), float('nan'))

        # Angle depends on the sign of b according to the paper's convention
        # Angle = phi0 + sgn(b) * t
        sign_b = copysign(1.0, b) # Get sign of b (+1.0 or -1.0)
        current_angle = phi0 + sign_b * t
        
        # Calculate displacement vector components
        delta_x = radial_factor * cos(current_angle)
        delta_y = radial_factor * sin(current_angle)

    # Calculate the absolute coordinates by adding displacement to the pole
    x = pw[0] + delta_x
    y = pw[1] + delta_y
    
    return (x, y)

# --- Main part of the script ---

# Parameters (User can change theta here)
# theta = pi / 3.0      # Original case (b > 0)
# theta = -pi / 3.0     # Test case (b < 0)
# theta = -pi / 2.0 + 0.01 # Test case (b < 0, close to 0)
theta = -pi / 2.0     # Test case (b = 0) -> Should result in a circle
# theta = 0.01          # Test case (b large positive) -> Rapid decay
# theta = -0.01         # Test case (b large negative) -> Rapid growth
# theta = 0.0           # Problematic case, safe_cot handles it, but interpretation needed


pw    = (10.0,  -8.0) # Point w (the pole of the spiral)
pv    = ( 6.0,   5.0) # Point v (the spiral starts here at t=0)

# --- Calculate initial conditions ---

# Calculate b using the safe cotangent function
# Check theta range first as per paper: theta in (-pi/2, pi/2)
# Note: safe_cot handles endpoints theta = +/- pi/2 correctly -> b=0
if not (-pi/2 <= theta <= pi/2):
    print(f"Error: theta = {theta:.4f} is outside the required range [-pi/2, pi/2].")
    # Exit or raise error if theta is strictly outside the valid range including endpoints
    # For now, let's stop execution
    pts = [] # Ensure pts is empty
    b = float('nan') # Indicate invalid b
else:
    if abs(theta) < 1e-9:
         print("Warning: theta is very close to 0. The spiral formula becomes ill-defined (b -> infinity).")
         print("The paper suggests a straight line S(t) = w + (v-w)(1-t) for theta=0.")
         # Handle this case separately if needed, maybe generate the straight line instead.
         # For now, proceed with calculation which might yield large b or NaN.
         
    b = safe_cot(theta)
    print(f"Input theta = {theta:.4f} radians")
    print(f"Calculated b = {b:.4f}")

    # Calculate vector u_wv = v - w (vector from pole to start)
    u_wv_x = pv[0] - pw[0]
    u_wv_y = pv[1] - pw[1]

    # Calculate L0 = ||v - w|| (initial distance)
    L0 = sqrt(u_wv_x**2 + u_wv_y**2)

    # Calculate phi0 = angle of u_wv (initial angle)
    phi0 = atan2(u_wv_y, u_wv_x)

    # --- Generate points for the spiral ---
    t_start = 0.0
    # Adjust t_end depending on whether spiral decays or grows
    # If b > 0 (decay), t can go higher.
    # If b < 0 (growth), t might need to be limited to avoid huge coordinates.
    # If b = 0 (circle), t represents angle swept.
    t_end = 8.0 # Default end time/angle
    if b < -1e-15: # If b is negative (growth)
        t_end = 4.0 # Limit t to avoid excessive growth, adjust as needed
    elif abs(b) < 1e-15: # If b is zero (circle)
        t_end = 2 * pi # Generate one full circle

    t_step = 0.05 # Use a smaller step for smoother curves
    pts = []
    t_current = t_start

    print(f"Generating points from t={t_start} to t={t_end} with step {t_step}...")

    while t_current <= t_end: # Use <= to include t_end point
        # Use the generalized function
        x_coord, y_coord = S_generalized(t_current, pw, L0, phi0, b) 
        
        # Check for NaN result (e.g., from overflow or invalid input)
        if math.isnan(x_coord) or math.isnan(y_coord):
            print(f"Stopping at t={t_current:.2f} due to invalid coordinates (NaN).")
            break
            
        pts.append((x_coord, y_coord))
        
        # Prevent infinite loop if t_step is zero or negative
        if t_step <= 0:
             print("Error: t_step must be positive.")
             break
        t_current += t_step
        # Safety break for very long loops
        if len(pts) > 10000:
            print("Warning: Exceeded 10000 points, stopping loop.")
            break


# --- Optional: Print some values for verification ---
if not math.isnan(b): # Only print if b was valid
    print(f"\nParameters:")
    print(f"theta = {theta:.4f} radians")
    print(f"b = cot(theta) = {b:.4f}")
    print(f"Pole w (pw) = {pw}")
    print(f"Start v (pv) = {pv}")
    print(f"Vector u_wv = ({u_wv_x:.4f}, {u_wv_y:.4f})")
    print(f"Initial distance L0 = ||u_wv|| = {L0:.4f}")
    print(f"Initial angle phi0 = {phi0:.4f} radians ({phi0 * 180/pi:.2f} degrees)")
    print(f"\nGenerated {len(pts)} points.")
    if pts:
        print(f"S(t=0.0): ({pts[0][0]:.4f}, {pts[0][1]:.4f}) (Should be close to pv {pv})")
        # Print intermediate point if enough points exist
        if len(pts) > 20:
           mid_index = len(pts) // 2
           mid_t = t_start + mid_index * t_step
           print(f"S(t={mid_t:.2f}): ({pts[mid_index][0]:.4f}, {pts[mid_index][1]:.4f})") 
        # Print last point
        if len(pts) > 1:
           last_t = t_start + (len(pts) - 1) * t_step
           print(f"S(t={last_t:.2f}) (last point): ({pts[-1][0]:.4f}, {pts[-1][1]:.4f})")



In [ ]:
#
# ConfluentSpiral class
# - Last cell but as a rolled up class
# - but still not correct for -theta values...
#
class ConfluentSpiral(object):
    #
    # Constructor
    #
    def __init__(self, pv, pw, theta=pi/3.0, t_inc=0.1, t_min=0.0, t_max=8.0):
        self.pv    = pv
        self.pw    = pw
        self.theta = theta
        self.t_inc = t_inc
        self.t_min = t_min
        self.t_max = t_max

        # Define cotangent function with safety checks
        def safe_cot(angle_rad):
            if abs(abs(angle_rad) - pi/2.0) < 1e-15: return 0.0
            tan_val = tan(angle_rad)
            if abs(tan_val) < 1e-15:
                print(f"Warning: theta={angle_rad:.4f} is very close to 0, cot(theta) approaches infinity.")
                return copysign(1e15, angle_rad) # cot(theta) has same sign as theta near 0
            return 1.0 / tan_val

        b      = safe_cot(theta)
        u_wv_x = pv[0] - pw[0]
        u_wv_y = pv[1] - pw[1]
        L0     = sqrt(u_wv_x**2 + u_wv_y**2)
        phi0   = atan2(u_wv_y, u_wv_x)

        self.pts = []
        t        = t_min
        while t <= t_max:
            # Case 1: b is effectively 0 (theta = +/- pi/2) -> Circle
            if abs(b) < 1e-15: # Use tolerance for floating point comparison
                current_angle = phi0 + t
                delta_x = L0 * cos(current_angle)
                delta_y = L0 * sin(current_angle)    
            # Case 2: b is non-zero -> Logarithmic Spiral
            else:
                try:
                    # Calculate the radial distance factor
                    radial_factor = L0 * exp(-b * t)
                except OverflowError:
                    # Handle cases where exp(-b*t) becomes too large (e.g., b<0 and t large)
                    print(f"Warning: Numerical overflow encountered for exp(-b*t) with b={b:.4f}, t={t:.4f}. Point set to NaN.")
                    return (float('nan'), float('nan'))
                # Angle depends on the sign of b according to the paper's convention
                # Angle = phi0 + sgn(b) * t
                sign_b = copysign(1.0, b) # Get sign of b (+1.0 or -1.0)
                current_angle = phi0 + sign_b * t
                # Calculate displacement vector components
                delta_x = radial_factor * cos(current_angle)
                delta_y = radial_factor * sin(current_angle)
            # Calculate the absolute coordinates by adding displacement to the pole
            x = pw[0] + delta_x
            y = pw[1] + delta_y
            self.pts.append((x,y))
            t += t_inc
    #
    # SVG Representation
    #
    def _repr_svg_(self):
        x0, y0, x1, y1 = self.pts[0][0], self.pts[0][1], self.pts[0][0], self.pts[0][1]
        for _pt_ in self.pts: x0, y0, x1, y1 = min(x0,_pt_[0]), min(y0,_pt_[1]), max(x1,_pt_[0]), max(y1,_pt_[1])
        x0, y0 = min(x0, min(self.pv[0], self.pw[0])), min(y0, min(self.pv[1], self.pw[1]))
        x1, y1 = max(x1, max(self.pv[0], self.pw[0])), max(y1, max(self.pv[1], self.pw[1]))
        x_perc = 0.05 * (x1 - x0)
        y_perc = 0.05 * (y1 - y0)
        x0, y0, x1, y1 = x0 - x_perc, y0 - y_perc, x1 + x_perc, y1 + y_perc
        ws, hs = 256, 256
        svg = [f'<svg x="0" y="0" width="{ws}" height="{hs}" viewBox="{x0} {y0} {x1-x0} {y1-y0}" xmlns="http://www.w3.org/2000/svg">']
        svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="#ffffff" />')
        x = int(x0)
        if int(x1) - int(x0) < 200:
            while x < x1:
                svg.append(f'<line x1="{x}" y1="{y0}" x2="{x}" y2="{y1}" stroke="#a0a0a0" stroke-width="0.05"/>')
                x += 1
        y = int(y0)
        if int(y1) - int(y0) < 200:
            while y < y1:
                svg.append(f'<line x1="{x0}" y1="{y}" x2="{x1}" y2="{y}" stroke="#a0a0a0" stroke-width="0.05"/>')
                y += 1
        svg.append(f'<line x1="{0.0}" y1="{y0}"  x2="{0.0}" y2="{y1}"  stroke="#a0a0a0" stroke-width="0.1"/>')
        svg.append(f'<line x1="{x0}"  y1="{0.0}" x2="{x1}"  y2="{0.0}" stroke="#a0a0a0" stroke-width="0.1"/>')
        svg.append(rt.svgText('pw', self.pw[0], self.pw[1]-0.5, txt_h=1.5, color='#a0a0a0', anchor='middle'))
        svg.append(f'<circle cx="{self.pw[0]}" cy="{self.pw[1]}" r="0.2" fill="#ff0000"/>')
        svg.append(rt.svgText('pv', self.pv[0], self.pv[1]+2.0, txt_h=1.5, color='#a0a0a0', anchor='middle'))
        _path_ = [f'M {self.pts[0][0]} {self.pts[0][1]}']
        for i in range(1, len(self.pts)): _path_.append(f'L {self.pts[i][0]} {self.pts[i][1]}')
        svg.append(f'<path d="{" ".join(_path_)}" fill="none" stroke="#000000" stroke-width="0.1"/>')
        svg.append(f'<circle cx="{self.pv[0]}" cy="{self.pv[1]}" r="0.2" fill="#ff0000"/>')
        for i in range(len(self.pts)):
            _pt_ = self.pts[i]
            svg.append(f'<circle cx="{_pt_[0]}" cy="{_pt_[1]}" r="0.1" fill="#000000"/>')
        svg.append('</svg>')
        return ''.join(svg)

_pv_, _pw_ = (1.0, 2.0),(-7.0, 9.0)
rt.tile([ConfluentSpiral(_pv_, _pw_, theta=-pi/3),
         ConfluentSpiral(_pv_, _pw_, theta=pi/3),
         ConfluentSpiral(_pv_, _pw_, theta=pi/4),
         ConfluentSpiral(_pv_, _pw_, theta=pi/5),
         ConfluentSpiral(_pv_, _pw_, theta=pi/6)], spacer=10)